# **Introduction:**

This file serves to design and test a custom implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS) within Keras. It is hoped that this custom function is able to be used like a standard Keras model, and will be trained and evaluated against the ANN designed previously. 

This ANFIS takes the load history, the distance to the task, and the total distance travelled thus far and performs inference about the suitability of a given robot for a task at hand. 

**Date Created:** 13/01/2025

**Date Modified:** 13/01/2025

# **Import Packages:** 

This section imports all the necessary packages for the ANFIS implementation. 

In [1]:
# import packages:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential 

# **Function & Class Definitions:**

Need to define numerous classes and functions for this implementation to work, as shown below. 

In [2]:
# define a function for triangular membership functions:
def triangular_mf(x, a, b, c):
    return tf.maximum(0.0, tf.minimum((x - a) / (b - a), (c - x) / (c - b)))

# custom fuzzification layer:
class FuzzificationLayer(layers.Layer):

    # constructor to initialize objects:
    def __init__(self, num_inputs, num_mfs):
        super(FuzzificationLayer, self).__init__()  # initialize in the same manner as the parent
        self.num_inputs = num_inputs                # assign the number of inputs
        self.num_mfs = num_mfs                      # assign the number of membership functions

        # create learnable mf parameters
        self.mf_params = self.add_weight(           
            shape = (num_inputs, num_mfs, 3),       # shape being num_inputs rows, num_mfs columns, and a depth of 3 for the 3 parameters in a triangular membership function
            initializer = 'random_uniform',         # randomly, uniformly initialize the weights
            trainable = True,                       # set to be trainable
        )

    # forward pass through layer given by:
    def call(self, inputs):
        mfs = []            # initialize empty list
        for i in range(self.num_inputs):        # for each input x_i
            mfs.append(                         # add the output to the mf list
                tf.stack(
                    # each input i across all instances is inputted, along with the parameters for the j-th membership function for the i-th input
                    [triangular_mf(inputs[:,i], *self.mf_params[i, j]) for j in range(self.num_mfs)], axis = 1
                )
            )
        return tf.concat(mfs, axis = 1) # return to user
    
# custom firing strength layer:
class FiringStrengthLayer(layers.Layer):
    def call(self, inputs):
        rules = tf.reduce_prod(inputs, axis = 1, keepdims = True)
        return rules
    
# custom normalization layer:
class NormalizationLayer(layers.Layer):
    def call(self, inputs):
        return inputs / tf.reduce_sum(inputs, axis = 1, keepdims = True)
    
# custom consequent layer:
class ConsequentLayer(layers.Layer):

    # constructor to initialize objects:
    def __init__(self, num_rules, num_inputs):
        super(ConsequentLayer, self).__init__()     # initialize in the same manner as the parent
        self.params = self.add_weight(              # takagi-sugeno consequent parameters
            shape = (num_rules, num_inputs + 1),
            initializer = 'random_uniform',
            trainable = True
        )

    # forward pass through layer given by:
    def call(self, inputs):
        # add a bias term to the inputs:
        bias = tf.ones((inputs.shape[0], 1))
        extended_inputs = tf.concat([inputs, bias], axis = 1)
        return tf.matmul(extended_inputs, self.params, transpose_b=True)
    
# custom output layer:
class OutputLayer(layers.Layer):
    def call(self, inputs):
        return tf.reduce_sum(inputs, axis=1, keepdims=True)

# **Define Testing Parameters:**

This section defines the testing parameters, such as the number of inputs, the number of membership functions, and the expected number of rules. This section also defines a function for creating and compiling Keras models using the custom layers.

Testing Parameters:

In [ ]:
# define parameters:
num_inputs = 3
num_mfs = 3
num_rules = num_mfs ** num_inputs

Model Generation Function:

In [ ]:
# function to make models:
def make_model(num_inputs, num_mfs, num_rules, rate):

    # instantiate model:
    model = Sequential()

    # add fuzzification layer:
    model.add